In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
os.chdir('/Users/andreasabia/Documents/ESCP/Consumer Insight & Analytics')
df=pd.read_csv('data_svedka(1).csv', sep=';')
df=df.loc[:,'BrandName':'total ad']

In [3]:
#Checking all expected variables are in the dataset + exploration
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 67 columns):
BrandName              263 non-null object
Brand ID               263 non-null int64
Year                   263 non-null int64
Absolut                263 non-null int64
Aristocrat             263 non-null int64
Barton                 263 non-null int64
Belvedere              263 non-null int64
Burnett                263 non-null int64
Chopin                 263 non-null int64
Crystal Palac          263 non-null int64
Finlandia              263 non-null int64
Fleischmann's          263 non-null int64
Fris                   263 non-null int64
Gilbey's               263 non-null int64
Gordon's               263 non-null int64
Grey Goose             263 non-null int64
Kamchatka              263 non-null int64
Ketel One              263 non-null int64
Level                  263 non-null int64
McCormick              263 non-null int64
Polar Ice              263 non-null int64
Popov       

In [4]:
# Dropping missing values
df=df.dropna()
# Converting the target column into numerical
df['diff'] = df['diff'].str.rstrip('%').astype('float') / 100.0
# Standardizing the target variable (0-1)
df['diff']=(df['diff']-df['diff'].min())/(df['diff'].max()-df['diff'].min())
# Standardizing other variables (normalized)
df['LagTotalMinusSales']=(df['LagTotalMinusSales']-df['LagTotalMinusSales'].mean())/df['LagTotalMinusSales'].std()

### Question 1
**Run a regression of the natural logarithm of change in sales on the natural logarithm of previous period’s prices. What is astonishing?**
<br>
An increase in price lead to an increase in sales. We can hence refer to the vodka as a Giffen good

In [5]:
target=df['diff']
q1_data=df['LnLPrice']
q1_data=sm.add_constant(q1_data)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [6]:
OLS_model=sm.OLS(target,q1_data)
result=OLS_model.fit()
print(result.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.026    
Dependent Variable: diff             AIC:                -460.7409
Date:               2020-05-04 16:57 BIC:                -453.6980
No. Observations:   250              Log-Likelihood:     232.37   
Df Model:           1                F-statistic:        7.656    
Df Residuals:       248              Prob (F-statistic): 0.00608  
R-squared:          0.030            Scale:              0.0091976
--------------------------------------------------------------------
               Coef.    Std.Err.     t      P>|t|    [0.025   0.975]
--------------------------------------------------------------------
const          0.0977     0.0329   2.9674   0.0033   0.0328   0.1625
LnLPrice       0.0212     0.0077   2.7670   0.0061   0.0061   0.0363
------------------------------------------------------------------
Omnibus:             277.308      Durbin-Watson:         0.977    
Pro

### Question 2
**To understand the influence of vodka quality, run a regression by adding tier 1 and tier 2 dummy variables.**
<br>
LnLPrice is now not significant anymore (p-value>0.05)

In [7]:
q2_data=df[['LnLPrice','Tier1','Tier2']]
q2_data=sm.add_constant(q2_data)

In [8]:
OLS_model=sm.OLS(target,q2_data)
result=OLS_model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     8.451
Date:                Mon, 04 May 2020   Prob (F-statistic):           2.29e-05
Time:                        16:57:47   Log-Likelihood:                 240.83
No. Observations:                 250   AIC:                            -473.7
Df Residuals:                     246   BIC:                            -459.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2254      0.046      4.877      0.0

### Question 3
**Add the natural log of marketing expenditures on print, outdoor and broadcasting. What happens?**
<br>
Tier 1 becomes non significant, although still very close to the threshold. 
<br>
LnMag is not significant.

In [9]:
q3_data=df[['LnLPrice','Tier1','Tier2','LnMag','LnNews','LnOut']]
q3_data=sm.add_constant(q3_data)

In [10]:
OLS_model=sm.OLS(target,q3_data)
result=OLS_model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     7.845
Date:                Mon, 04 May 2020   Prob (F-statistic):           9.83e-08
Time:                        16:57:47   Log-Likelihood:                 250.70
No. Observations:                 250   AIC:                            -487.4
Df Residuals:                     243   BIC:                            -462.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2130      0.046      4.600      0.0

### Question 4
**To understand the influence of competition, run a regression by adding the sum of sales of all the competing brands in the previous year (“lagtotalminussales”) to the set of independent variables.**
<br>

In [11]:
q4_data=df[['LnLPrice','Tier1','Tier2','LnMag','LnNews','LnOut','LagTotalMinusSales']]
q4_data=sm.add_constant(q4_data)

In [12]:
OLS_model=sm.OLS(target,q4_data)
result=OLS_model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     8.823
Date:                Mon, 04 May 2020   Prob (F-statistic):           1.15e-09
Time:                        16:57:47   Log-Likelihood:                 256.98
No. Observations:                 250   AIC:                            -498.0
Df Residuals:                     242   BIC:                            -469.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.2481      0

### Question 5
**To measure the sales growth of new brands compared with the existent ones, include the variables “Firstintro” to the variable set.**
<br>

In [13]:
q5_data=df[['LnLPrice','Tier1','Tier2','LnMag','LnNews','LnOut','LagTotalMinusSales','Firstintro']]
q5_data=sm.add_constant(q5_data)

In [14]:
OLS_model=sm.OLS(target,q5_data)
result=OLS_model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     18.61
Date:                Mon, 04 May 2020   Prob (F-statistic):           1.19e-21
Time:                        16:57:47   Log-Likelihood:                 288.70
No. Observations:                 250   AIC:                            -559.4
Df Residuals:                     241   BIC:                            -527.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3322      0

### Question 6
**Why does the coefficient for price and advertising change?**
<br>
Perhaps, the effect of tier 1 and tier 2 features change the effect the price would have on the change in sales. Indeed, for a higher tiered Vodka, we are willing to pay more, but within the tiers, perhaps the customers are not willing to pay more.

### Question 7
**What recommendations do you have for the producer of Svedka?**
<br>
They should watch out from the new entrants in the market, who are apparently growing in sales.
<br>
They should focus their marketing spend on magazine advertisements.
<br>
They should be careful with increasing their price, as it leads to a decrease in sales.